**Задача: обучить бинарный классификатор для поиска токсичного контента (твитов)**

**Имеются: 2 датасета **

**train_data.csv - обучающая выборка**

**test_data.csv - тестовая выборка**

In [ ]:
import glob
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


## **1. Загрузка нужных библиотек**

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# библиотека, где реализованы основные алгоритмы машинного обучения
from sklearn.metrics import * 
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline

import nltk
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pymystem3 import Mystem

## **2. Загрузка данных**

In [ ]:
train_data = pd.read_csv("/gdrive/My Drive/nlp_tcc_train_data.csv")
test_data = pd.read_csv("/gdrive/My Drive/nlp_tcc_test_data.csv")

In [ ]:
train_data.head()

,comment,toxic
0,Преступление и наказание\n,0.0
1,"И именно эти неработающие весы показывают, что...",0.0
2,"В Японии такие панельки, ебанько.\n",0.0
3,Еще у нас выявляют трещины с помощью белой кра...,0.0
4,"Дочитал до поезда в Норильск , дальше не стал\n",0.0


In [ ]:
test_data.head()

,comment_id,comment
0,0,"В данном случае не пионервожатая, а техничка, ..."
1,1,В данном посте рассматривается обычный ленточн...
2,2,"Как я понимаю, у поверхностей отражаемость пог..."
3,3,Австралийские пауки съедят все живое на планете))
4,4,У нас раньше Амур пиво целые бутылки принимал ...


In [ ]:
y_train = train_data['toxic']
y_train

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
10804    0.0
10805    1.0
10806    1.0
10807    0.0
10808    0.0
Name: toxic, Length: 10809, dtype: float64

## **3. Масштабирование данных с помощью tf-idf**

In [ ]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

## **4. Деление данных на подвыборки**

In [ ]:
%%time
# Чтобы посчитать TF-IDF для корпуса текстов, вызовем функцию fit_transform():
tf_idf_train = count_tf_idf.fit_transform(train_data['comment'])
tf_idf_test = count_tf_idf.transform(test_data['comment'])

CPU times: user 923 ms, sys: 9.98 ms, total: 933 ms
Wall time: 1.53 s


In [ ]:
print("Размер матрицы:", tf_idf_train.shape)
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (10809, 56413)
Размер матрицы: (3603, 56413)


## **5. Обучение логистической регрессии**

In [ ]:
logreg = LogisticRegression(random_state=5)

# param_grid = [{"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}]
param_grid = [
    {'C': [0.01, 0.1, 1, 2, 10, 100], 'penalty': ['l1', 'l2']}
]
# =109.85411419875572, class_weight=None, dual=False,
#           fit_intercept=True, intercept_scaling=1, max_iter=100,
#           multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
#           solver='warn', tol=0.0001, verbose=0, warm_start=False

logreg_grid = GridSearchCV(logreg, param_grid, cv=5, verbose=True, n_jobs=-1)

logreg_grid.fit(tf_idf_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueError: Solver l

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=5), n_jobs=-1,
             param_grid=[{'C': [0.01, 0.1, 1, 2, 10, 100],
                          'penalty': ['l1', 'l2']}],
             verbose=True)

## **6. Предсказание на тестовой выборке**

In [17]:
predictions = logreg_grid.predict(tf_idf_test)
print("Наилучшее значение проверки: {:.5f}".format(logreg_grid.best_score_))

Наилучшее значение проверки: 0.84411


In [ ]:
submission = pd.DataFrame({'toxic':predictions})
submission.to_csv('predictions', index=True)